## Implementing the perceptron learning algorithm

This example shows how to implement the perceptron learning algorithm using NumPy. We implement the methods `fit` and `predict` so that our classifier can be used in the same way as any scikit-learn classifier. (See the [scikit-learn documentation](http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html).)

The code uses a little bit of object-oriented programming. It's not anything particularly complicated, but if you're not used to object-oriented programming in Python, you might take a look at [this tutorial](https://python.swaroopch.com/oop.html).

In [1]:
import numpy as np

We first create a class that represents linear classifiers in general. This class does not have a `fit` method, because that will be implemented by subclasses representing specific learning algorithms for linear classifiers, e.g. the perceptron. So the thing we need to do here is to implement the `predict` method, because prediction works identically for all linear classifiers, regardless of how they were trained.

We also include a helper method `find_classes`, which finds the two output classes and associates them with positive and negative classifier scores, respectively.

In [2]:
class LinearClassifier(object):
    
    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of classes
        is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[0]
        self.negative_class = classes[1]
    
    def predict(self, X):        
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be stored in
        a matrix, where each row contains the features for one instance.
        """

        # First compute the output scores
        scores = X.dot(self.w)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores>=0.0, scores<0.0], 
                        [self.positive_class, 
                         self.negative_class])
        return out

We now write the class that implements the perceptron learning algorithm. The actual learning algorithm is in the method called `fit`.

Note that this class has the same name as the `Perceptron` class in scikit-learn, so be careful when you import so that you don't get a name clash.

In [3]:
class Perceptron(LinearClassifier):
    
    def __init__(self, n_iter=10):
        """
        The constructor can optionally take a parameter n_iter specifying how 
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """
        
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros( n_features )

        for i in range(self.n_iter):            
            for x, y in zip(X, Y):
                
                # Compute the output score for this instance.
                score = x.dot(self.w)    

                # If there was an error, update the weights.
                if y == self.positive_class and score <= 0:
                    self.w += x
                if y == self.negative_class and score >= 0:
                    self.w -= x

### Testing our perceptron implementation on the Adult dataset

We will now test our perceptron implementation on the Adult dataset. This example reuses some code from the first computer exercise, to process the format of the dataset. To use this dataset, you need to download the files `adult.names`, `adult.data`, and `adult.test` from [the UCI machine learning repository](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/).

In [4]:
def read_names(filename):
    names = []
    types = []
    with open(filename) as f:
        for l in f:
            if l[0] == '|' or ':' not in l:
                continue
            cols = l.split(':')
            names.append(cols[0])
            if cols[1].startswith(' continuous.'):
                types.append(float)
            else:
                types.append(str)
    return names, types

col_names, col_types = read_names('adult.names')

def read_data(filename, col_names, col_types):
    X = []
    Y = []
    with open(filename) as f:
        for l in f:
            cols = l.strip('\n.').split(', ')
            if len(cols) < len(col_names):
                continue
            X.append( { n:t(c) for n, t, c in zip(col_names, col_types, cols) } )
            Y.append(cols[-1])
    return X, Y

# read the training set
Xtrain, Ytrain = read_data('adult.data', col_names, col_types)

# read the test set
Xtest, Ytest = read_data('adult.test', col_names, col_types)

To exemplify the instances in this dataset, let's print the input and output for the first instance. The input consists of a feature dictionary, containing named attributes such as `age`, `education` etc. The output is a string: in this case, either `'<=50K'` (low earner) or `'>50K'` (high earner). This is a *binary* classification problem because we have two output classes.

In [5]:
from pprint import pprint

pprint(Xtrain[0])
print()
print(Ytrain[0])

{'age': 39.0,
 'capital-gain': 2174.0,
 'capital-loss': 0.0,
 'education': 'Bachelors',
 'education-num': 13.0,
 'fnlwgt': 77516.0,
 'hours-per-week': 40.0,
 'marital-status': 'Never-married',
 'native-country': 'United-States',
 'occupation': 'Adm-clerical',
 'race': 'White',
 'relationship': 'Not-in-family',
 'sex': 'Male',
 'workclass': 'State-gov'}

<=50K


Now let's assemble the building blocks.

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# The DictVectorizer is used to map symbolic features to numerical vectors.
# Note that we set sparse=False, because our Perceptron implementation assumes
# that the examples are stored in a NumPy matrix.
dv = DictVectorizer(sparse=False)

# A StandardScaler divides the features by their standard deviation. The purpuse is that
# the numerical features should have a similar magnitude.
sc = StandardScaler(with_mean=False)

# Make an instance of the perceptron class we implemented above.
clf = Perceptron()

# Combine the vectorizer, scaler and the classifier into a pipeline.
pipeline = make_pipeline( dv, sc, clf )

# Train the classifier, evaluate on the test set.
pipeline.fit(Xtrain, Ytrain);

And finally run the classifier on the test set and compute its accuracy.

In [7]:
Yguess = pipeline.predict(Xtest)
accuracy_score(Ytest, Yguess)

0.7913956978489245

### Inspecting the features learned by the classifier

Now, let's take a look at what the perceptron algorithm has come up with. First, let's see which category corresponds to the positive scores, and which to the negative scores.

In [8]:
clf.positive_class, clf.negative_class

('<=50K', '>50K')

This means that positive scores will be interpreted as the category `<=50K`, and negative scores as `>50K`.

We will then see which features the learning algorithm has assigned high weights to. We can first just look at the weights stored in the weight vector `w`, that we built in the `fit` method that we created previously. This is a long vector, so we'll just print the first 10 dimensions. We see that the first three features have negative weights, which shows that an increase in these features will increase our certainty that this person is a high earner (negative classifier score). The other seven features point in the other direction: increasing them makes the classifier think that this person is a low earner.

In [9]:
clf.w[:10]

array([ -19.97589562, -101.36763558,  -24.22800453,   14.09665603,
         24.10751822,   14.57480801,   10.39962876,    0.        ,
         10.39962876,   22.16560518])

The result above didn't tell us that much, really, because it's not obvious how to interpret the positions. To understand the meaning of each position, we need to look into the `DictVectorizer` that we used to map named features into a feature matrix.

In a `DictVectorizer`, this information is stored in the attribute called `feature_names_`. The feature names appear in the same order as they do in the weight vector. So this means that the first column in the feature matrix is `age`. The second feature, `capital-gain`, has a much stronger association with the negative class.

In [10]:
dv.feature_names_[:10]

['age',
 'capital-gain',
 'capital-loss',
 'education-num',
 'education=10th',
 'education=11th',
 'education=12th',
 'education=1st-4th',
 'education=5th-6th',
 'education=7th-8th']

We print the 20 features that have the highest negative weights. In this case, the negative class is `>50K`, or the people who earned more than $50,000 a year. As you can see, features look quite meaningful: for instance, people who own capital or have a college degree are more likely to have a high income.

(If you wonder about the functions `sorted` and `zip`, please take a look at the [documentation of Python built-in functions](https://docs.python.org/3/library/functions.html).)

In [11]:
for weight, fname in sorted( zip(clf.w, dv.feature_names_) )[:20]:
    print(fname, weight)

capital-gain -101.36763557732267
native-country=Cambodia -38.7427498651224
native-country=Italy -36.569732969241024
marital-status=Married-civ-spouse -34.13648500928391
education=Masters -25.951043559299272
capital-loss -24.22800452508256
occupation=Tech-support -23.448415270421496
native-country=Iran -22.38307405179305
native-country=Jamaica -22.383074051792764
age -19.975895615583973
native-country=England -17.34944795898751
hours-per-week -16.194567820428155
occupation=Exec-managerial -15.277167174810899
native-country=Canada -13.729725486341648
relationship=Not-in-family -13.53588551764624
occupation=Prof-specialty -12.207389010415373
education=Assoc-acdm -11.600057330203043
fnlwgt -10.122362093930322
education=Doctorate -10.050378152592227
relationship=Wife -9.00791242218654


Conversely, the features most strongly associated with the positive class (`<=50K`, low earners) also tend to be meaningful, such as being unemployed or not having an education.

In [12]:
for weight, fname in sorted( zip(clf.w, dv.feature_names_), reverse=True)[:20]:
    print(fname, weight)

native-country=Laos 54.7813867426214
native-country=Ecuador 54.7813867426214
native-country=Yugoslavia 54.78138674262097
native-country=Peru 54.78138674262073
native-country=Scotland 54.78138674262064
relationship=Own-child 53.45636183421069
occupation=Armed-Forces 38.742749865123166
native-country=Thailand 38.742749865123
native-country=Trinadad&Tobago 38.74274986512226
native-country=Vietnam 38.74274986512215
marital-status=Never-married 34.088785554798896
native-country=Columbia 31.63859985841679
marital-status=Married-AF-spouse 31.638599858416644
occupation=Farming-fishing 30.30804031055438
sex=Female 29.922553484768287
native-country=Nicaragua 27.404403571569752
marital-status=Married-spouse-absent 25.23960880504893
native-country=South 24.515335393364108
native-country=Haiti 24.515335393363735
education=9th 24.41509104406004
